# Charts

In [2]:
import numpy as np
import pandas as pd
from siuba import *
from calitp import *
import intake
import data_prep
import _utils
import shared_utils
import altair as alt
import altair_saver
from shared_utils import altair_utils 
pd.options.display.max_rows = 100
pd.options.display.float_format = "{:.0f}".format

In [3]:
BC_GTFS_NTD2 = pd.read_parquet('BC_GTFS_NTD.parquet')

In [4]:
#Grabbing only one row for each agency.
Aggregate1 = BC_GTFS_NTD2.groupby(['organization_name','reporter_type','GTFS', 'fleet_size', 'Is_Agency_In_BC_Only_1_means_Yes']).agg({'total_vehicles':'max',
       'average_age_of_fleet__in_years_':'max','doors_sum':'max','_0_9':'max', '_10_12':'max', '_13_15':'max', '_16_20':'max', '_21_25':'max',
       '_26_30':'max', '_31_60':'max', '_60plus':'max',})

In [5]:
#Reset index
Aggregate1 = Aggregate1.reset_index()


#  GTFS Status Charts
* One for vehicles over 9
* One for vehicles over 15

In [7]:
vehicles_gtfs = _utils.aggregation_one(Aggregate1, 'GTFS')
vehicles_gtfs

,GTFS,vehicles_older_than_9,vehicles_older_than_15,vehicles_0_to_9,vehicles_percent_older_than_9,vehicles_percent_older_than_15,vehicles_percent_0_to_9
0,Long-term solution needed,7,1,24,0,0,0
1,Needed,34,7,122,2,1,2
2,None,51,5,191,3,1,4
3,Ok,1901,720,4470,93,96,90
4,Research,45,16,151,2,2,3


### Looking at vehicles 9+ by GTFS
![pic](./chart_outputs/bar_GTFS_by_vehicles_older_than_9.png)

In [8]:
_utils.basic_bar_chart(vehicles_gtfs,'GTFS','vehicles_older_than_9','GTFS') 

alt.Chart(...)

In [9]:
_utils.basic_bar_chart(vehicles_gtfs,'GTFS','vehicles_percent_older_than_9','GTFS') 

alt.Chart(...)

### Looking at vehicles 15+ by GTFS
![pic](./chart_outputs/bar_GTFS_by_vehicles_older_than_15.png)

In [10]:
_utils.basic_bar_chart(vehicles_gtfs,'GTFS','vehicles_older_than_15', 'GTFS') 

alt.Chart(...)

# Fleet size 

In [11]:
fleet = _utils.aggregation_one(Aggregate1,'fleet_size') 

In [12]:
# don't need no info
fleet =  fleet.loc[fleet['fleet_size'] != 'No Info'] 

### Vehicles older than 9
![pic](./chart_outputs/bar_fleet_size_by_vehicles_older_than_9.png)

In [13]:
_utils.basic_bar_chart(fleet,'fleet_size','vehicles_older_than_9', 'fleet_size') 

alt.Chart(...)

### Vehicles older than 15
![pic](./chart_outputs/bar_fleet_size_by_vehicles_older_than_15.png)

In [14]:
_utils.basic_bar_chart(fleet,'fleet_size','vehicles_older_than_15', 'fleet_size') 

alt.Chart(...)

# Other
## Looking at organizations & GTFS
![pic](./chart_outputs/bar_GTFS_by_count_of_agencies.png)

In [15]:
Orgs = Aggregate1.groupby(['GTFS']).agg({'organization_name':'nunique'})
Orgs = Orgs.reset_index()
Orgs = Orgs.rename(columns = {'organization_name': 'count_of_agencies'}) 
_utils.basic_bar_chart(Orgs,'GTFS','count_of_agencies', 'GTFS') 

alt.Chart(...)

## Looking at reporter type & GTFS
![testing](./chart_outputs/bar_GTFS_by_count_of_agencies.png)

In [16]:
Reporter_type_agg = Aggregate1.groupby(['reporter_type']).agg({'organization_name':'nunique'})
Reporter_type_agg = Reporter_type_agg.reset_index()
Reporter_type_agg = Reporter_type_agg.rename(columns = {'organization_name':'Count of Agencies'})
_utils.basic_bar_chart(Reporter_type_agg,'reporter_type','Count of Agencies', 'reporter_type') 

alt.Chart(...)

## Looking at agencies that spent the most..
![pic](./chart_outputs/bar_organization_name_by_allocationamount.png)

In [17]:
Most_Money = BC_GTFS_NTD2.groupby(['organization_name','GTFS']).agg({'allocationamount':'sum'})

In [18]:
Most_Money= Most_Money.sort_values('allocationamount').tail(10)
Most_Money = Most_Money.reset_index()

In [19]:
_utils.basic_bar_chart(Most_Money,'organization_name','allocationamount','organization_name') 

alt.Chart(...)